In [1]:
# import libraries
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import pandas as pd

## Obtaining GTZAN dataset

We will be using `deeplake` to get the GTZAN dataset

In [3]:
from IPython.display import clear_output
# deep lake is not part of google colab by default
!pip3 install deeplake

clear_output()

In [ ]:
# setting up the deeplake environment, so that we could download it
import os
os.environ['DEEPLAKE_DOWNLOAD_PATH']='./'

In [8]:
import deeplake
ds = deeplake.load("hub://activeloop/gtzan-genre", access_method="local")

/

Opening dataset in read-only mode as you don't have write permissions.


Copying dataset: 0%|          | 0/102 [00:00<?
Copying dataset: 1%|          | 1/102 [00:00<01:33
Copying dataset: 2%|▏         | 2/102 [00:02<01:51
Copying dataset: 3%|▎         | 3/102 [00:03<01:55
Copying dataset: 4%|▍         | 4/102 [00:04<02:00
Copying dataset: 5%|▍         | 5/102 [00:05<01:57
Copying dataset: 6%|▌         | 6/102 [00:06<01:52
Copying dataset: 7%|▋         | 7/102 [00:08<01:50
Copying dataset: 8%|▊         | 8/102 [00:09<01:49
Copying dataset: 9%|▉         | 9/102 [00:10<01:54
Copying dataset: 10%|▉         | 10/102 [00:12<02:00
Copying dataset: 11%|█         | 11/102 [00:13<02:07
Copying dataset: 12%|█▏        | 12/102 [00:15<02:13
Copying dataset: 13%|█▎        | 13/102 [00:17<02:18
Copying dataset: 14%|█▎        | 14/102 [00:18<02:14
Copying dataset: 15%|█▍        | 15/102 [00:20<02:14
Copying dataset: 16%|█▌        | 16/102 [00:22<02:21
Copying dataset: 17%|█▋        | 17/102 [00:23<02:14
Copying dataset: 18%|█▊        | 18/102 [00:25<02:11
Copying dataset: 